In [4]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl
import yfinance as yf


plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [40]:
aapl = yf.Ticker("AAPL")
aapl_info = aapl.info

In [41]:
aapl = aapl.history(start='2010-01-01')
aapl.reset_index(inplace = True)
aapl.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits'],
      dtype='object')

In [42]:
# apple_share_price_data.index = pd.to_datetime(apple_share_price_data.index)
aapl.Date = pd.to_datetime(aapl.Date, format='%Y/%m/%d')
aapl.drop(columns=['Dividends','Stock Splits'],inplace= True)
aapl

,Date,Open,High,Low,Close,Volume
0,2009-12-31,6.508215,6.514933,6.429736,6.434927,352410800
1,2010-01-04,6.517373,6.550047,6.485310,6.535084,493729600
2,2010-01-05,6.553102,6.583333,6.511877,6.546384,601904800
3,2010-01-06,6.546383,6.572340,6.435536,6.442255,552160000
4,2010-01-07,6.466073,6.473707,6.383625,6.430345,477131200
...,...,...,...,...,...,...
3151,2022-07-11,145.669998,146.639999,143.779999,144.869995,63141600
3152,2022-07-12,145.759995,148.449997,145.050003,145.860001,77588800
3153,2022-07-13,142.990005,146.449997,142.119995,145.490005,71185600
3154,2022-07-14,144.080002,148.949997,143.250000,148.470001,77996900


In [43]:
aapl.set_index(aapl.Date, inplace= True)
aapl.drop(columns=['Date'],inplace=True)
aapl

,Open,High,Low,Close,Volume
Date,,,,,
2009-12-31,6.508215,6.514933,6.429736,6.434927,352410800
2010-01-04,6.517373,6.550047,6.485310,6.535084,493729600
2010-01-05,6.553102,6.583333,6.511877,6.546384,601904800
2010-01-06,6.546383,6.572340,6.435536,6.442255,552160000
2010-01-07,6.466073,6.473707,6.383625,6.430345,477131200
...,...,...,...,...,...
2022-07-11,145.669998,146.639999,143.779999,144.869995,63141600
2022-07-12,145.759995,148.449997,145.050003,145.860001,77588800
2022-07-13,142.990005,146.449997,142.119995,145.490005,71185600


In [45]:
def get_stoch_osc(high, low, close, k_lookback, d_lookback):
    lowest_low = low.rolling(k_lookback).min()
    highest_high = high.rolling(k_lookback).max()
    k_line = ((close - lowest_low) / (highest_high - lowest_low)) * 100
    d_line = k_line.rolling(d_lookback).mean()
    return k_line, d_line

aapl['%k'], aapl['%d'] = get_stoch_osc(aapl['High'], aapl['Low'], aapl['Close'], 14, 3)
aapl.tail()

,Open,High,Low,Close,Volume,%k,%d
Date,,,,,,,
2022-07-11,145.669998,146.639999,143.779999,144.869995,63141600,81.166488,92.383130
2022-07-12,145.759995,148.449997,145.050003,145.860001,77588800,82.356964,86.882846
2022-07-13,142.990005,146.449997,142.119995,145.490005,71185600,79.836560,81.120004
2022-07-14,144.080002,148.949997,143.250000,148.470001,77996900,96.837971,86.343832
2022-07-15,149.779999,150.860001,148.199997,149.232498,43644244,90.476871,89.050467


In [47]:
# MACD CALCULATION

def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist

aapl['macd'] = get_macd(aapl['Close'], 26, 12, 9)[0]
aapl['macd_signal'] = get_macd(aapl['Close'], 26, 12, 9)[1]
aapl['macd_hist'] = get_macd(aapl['Close'], 26, 12, 9)[2]
aapl = aapl.dropna()
aapl.tail()


,Open,High,Low,Close,Volume,%k,%d,macd,macd_signal,macd_hist
Date,,,,,,,,,,
2022-07-11,145.669998,146.639999,143.779999,144.869995,63141600,81.166488,92.383130,0.073459,-1.458203,1.531662
2022-07-12,145.759995,148.449997,145.050003,145.860001,77588800,82.356964,86.882846,0.372747,-1.092013,1.464760
2022-07-13,142.990005,146.449997,142.119995,145.490005,71185600,79.836560,81.120004,0.573469,-0.758917,1.332386
2022-07-14,144.080002,148.949997,143.250000,148.470001,77996900,96.837971,86.343832,0.961915,-0.414750,1.376665
2022-07-15,149.779999,150.860001,148.199997,149.232498,43644244,90.476871,89.050467,1.316117,-0.068577,1.384694


In [48]:
# TRADING STRATEGY

def implement_stoch_macd_strategy(prices, k, d, macd, macd_signal):    
    buy_price = []
    sell_price = []
    stoch_macd_signal = []
    signal = 0

    for i in range(len(prices)):
        if k[i] < 30 and d[i] < 30 and macd[i] < -2 and macd_signal[i] < -2:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)
                
        elif k[i] > 70 and d[i] > 70 and macd[i] > 2 and macd_signal[i] > 2:
            if signal != -1 and signal != 0:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_macd_signal.append(0)
            
    return buy_price, sell_price, stoch_macd_signal
            
buy_price, sell_price, stoch_macd_signal = implement_stoch_macd_strategy(aapl['Close'], aapl['%k'], aapl['%d'], aapl['macd'], aapl['macd_signal'])

In [49]:
# POSITION

position = []
for i in range(len(stoch_macd_signal)):
    if stoch_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(aapl['Close'])):
    if stoch_macd_signal[i] == 1:
        position[i] = 1
    elif stoch_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
close_price = aapl['Close']
k_line = aapl['%k']
d_line = aapl['%d']
macd_line = aapl['macd']
signal_line = aapl['macd_signal']
stoch_macd_signal = pd.DataFrame(stoch_macd_signal).rename(columns = {0:'stoch_macd_signal'}).set_index(aapl.index)
position = pd.DataFrame(position).rename(columns = {0:'stoch_macd_position'}).set_index(aapl.index)

frames = [close_price, k_line, d_line, macd_line, signal_line, stoch_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy

,Close,%k,%d,macd,macd_signal,stoch_macd_signal,stoch_macd_position
Date,,,,,,,
2010-01-25,6.201018,32.067413,23.273455,-0.047727,-0.017703,0,1
2010-01-26,6.288658,47.743503,27.670715,-0.048264,-0.023815,0,1
2010-01-27,6.347897,58.292517,46.034477,-0.043409,-0.027734,0,1
2010-01-28,6.085590,11.582431,39.206150,-0.060035,-0.034194,0,1
2010-01-29,5.864812,7.154154,25.676367,-0.089989,-0.045353,0,1
...,...,...,...,...,...,...,...
2022-07-11,144.869995,81.166488,92.383130,0.073459,-1.458203,0,1
2022-07-12,145.860001,82.356964,86.882846,0.372747,-1.092013,0,1
2022-07-13,145.490005,79.836560,81.120004,0.573469,-0.758917,0,1


In [50]:
# BACKTESTING

aapl_ret = pd.DataFrame(np.diff(aapl['Close'])).rename(columns = {0:'returns'})
stoch_macd_strategy_ret = []

for i in range(len(aapl_ret)):
    try:
        returns = aapl_ret['returns'][i] * strategy['stoch_macd_position'][i]
        stoch_macd_strategy_ret.append(returns)
    except:
        pass
    
stoch_macd_strategy_ret_df = pd.DataFrame(stoch_macd_strategy_ret).rename(columns = {0:'stoch_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / aapl['Close'][0])
stoch_macd_investment_ret = []

for i in range(len(stoch_macd_strategy_ret_df['stoch_macd_returns'])):
    returns = number_of_stocks * stoch_macd_strategy_ret_df['stoch_macd_returns'][i]
    stoch_macd_investment_ret.append(returns)

stoch_macd_investment_ret_df = pd.DataFrame(stoch_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(stoch_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the STOCH MACD strategy by investing $100k in AAPL : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the STOCH MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the STOCH MACD strategy by investing $100k in AAPL : 1821611.6
Profit percentage of the STOCH MACD strategy : 1821%


In [59]:
# SPY ETF COMPARISON

def get_benchmark(start_date, investment_value):
    spy = yf.Ticker("SPY")
    spy = spy.history(start='2010-01-01')['Close']
    benchmark = pd.DataFrame(np.diff(spy)).rename(columns = {0:'benchmark_returns'})
    
    investment_value = investment_value
    number_of_stocks = floor(investment_value/spy[0])
    benchmark_investment_ret = []
    
    for i in range(len(benchmark['benchmark_returns'])):
        returns = number_of_stocks*benchmark['benchmark_returns'][i]
        benchmark_investment_ret.append(returns)

    benchmark_investment_ret_df = pd.DataFrame(benchmark_investment_ret).rename(columns = {0:'investment_returns'})
    return benchmark_investment_ret_df

benchmark = get_benchmark('2010-01-01', 100000)

investment_value = 100000
total_benchmark_investment_ret = round(sum(benchmark['investment_returns']), 2)
benchmark_profit_percentage = floor((total_benchmark_investment_ret/investment_value)*100)
print(cl('Benchmark profit by investing $100k : {}'.format(total_benchmark_investment_ret), attrs = ['bold']))
print(cl('Benchmark Profit percentage : {}%'.format(benchmark_profit_percentage), attrs = ['bold']))
print(cl('STOCH MACD Strategy profit is {}% higher than the Benchmark Profit'.format(profit_percentage - benchmark_profit_percentage), attrs = ['bold']))


Benchmark profit by investing $100k : 337625.37
Benchmark Profit percentage : 337%
STOCH MACD Strategy profit is 1484% higher than the Benchmark Profit
